<div align="center"><img src="./images/DLI_Header.png"></div>

# Monte Carlo Approximation of $\pi$ - NVSHMEM

In this notebook we will introduce NVSHMEM and make a first pass of using it in the monte-carlo approximation of $\pi$ program.

## Objectives

By the time you complete this notebook you will:

- Understand the benefits of using NVSHMEM for multi-GPU applications.
- Be able to write, compile, and run an NVSHMEM program that utilizes multiple GPUs.

## NVSHMEM

[NVSHMEM](https://developer.nvidia.com/nvshmem) is a parallel programming model for efficient and scalable communication across multiple NVIDIA GPUs. NVSHMEM, which is based on [OpenSHMEM](http://openshmem.org/site/), provides a global address space for data that spans the memory of multiple GPUs and can be accessed with fine-grained GPU-initiated operations, CPU-initiated operations, and operations on CUDA streams. NVSHMEM offers a compelling multi-GPU programming model for many application use cases, and is especially valuable on modern GPU servers that have a high density of GPUs per server node and complex interconnects such as [NVIDIA NVSwitch](https://www.nvidia.com/en-us/data-center/nvlink/) on the [NVIDIA DGX A100 server](https://www.nvidia.com/en-us/data-center/dgx-a100/).

<center><img src="images/NVSHMEM.png" width="1000"></center>

## Motivation for NVSHMEM

Traditionally, communication patterns involving GPUs on multiple servers may look like the following: <span style="color:limegreen">compute</span> happens on the GPU, while <span style="color:skyblue">communication</span> happens on the CPU after synchronizing the GPU (to ensure that the data we send is valid). While this approach is very easy to program, it inserts the latency of initiating the communication and/or launching the kernel on the application's critical path. We are losing out on the ability to overlap communication with compute. If we do overlap communication with compute by pipelining the work, we can partially hide the latency, but at the cost of increased application complexity.

<center><img src="images/CPU_initiated_communication.png" width="1000"/></center>

By contrast, in a model with GPU-initiated rather than CPU-initiated communication, we do *both* compute and communication directly from the GPU. We can write extremely fine-grained communication patterns this way, and we can hide communication latency by the very nature of the GPU architecture (where warps that are computing can continue on while other warps are stalled waiting for data).

<center><img src="images/GPU_initiated_communication.png" width="1000"></center>

## Launching NVSHMEM Applications

NVSHMEM, like MPI, is an example of the SPMD programming style. NVSHMEM provides a launcher script[<sup>1</sup>](#footnote1) called `nvshmrun` that handles launching the $M$ processes. The arguments to `nvshmrun` are `-np`, the number of processes to launch, and then the application executable followed by any arguments to that executable. Each independent process is called a **Processing Element (PE)** and has a unique (zero-indexed) numerical identifier associated with it[<sup>2</sup>](#footnote2).

<center><img src="images/nvshmrun.png" width="1000"></center>

## Using NVSHMEM in Code

Let's learn the mechanics of launching multiple processes with NVSHMEM in application code.

### Initializing and Finalizing NVSHMEM

As a core requirement on the host side, we must initialize and finalize NVSHMEM as the first and last things in our program.

```cpp
nvshmem_init();
...
nvshmem_finalize();
```

### Obtaining Processing Element IDs

The API call [nvshmem_my_pe()](https://docs.nvidia.com/nvshmem/api/gen/api/setup.html#nvshmem-my-pe) returns the unique numerical ID of each PE.

```cpp
int my_pe = nvshmem_my_pe();
int device = my_pe;
cudaSetDevice(device);
```

In multi-node environments you will have to account for the fact that CUDA devices are always zero-indexed within each node. In that case you would obtain the PE identifier *local* to that node. For example, if we were using two nodes with four GPUs each, then we would ask our job launcher to run four tasks per node (e.g. `nvshmrun -np 8 -ppn 4 -hosts hostname1,hostname2`) and then do[<sup>3</sup>](#footnote3):

```cpp
int my_pe_node = nvshmem_team_my_pe(NVSHMEMX_TEAM_NODE);
int device = my_pe_node;
cudaSetDevice(device);
```

### Compiling NVSHMEM Code

[Compiling](https://docs.nvidia.com/nvshmem/api/using.html#compiling-nvshmem-programs) looks similar to before, but we now need to point to the relevant include and library directories for NVSHMEM (`-I $NVSHMEM_HOME/include -L $NVSHMEM_HOME/lib -lnvshmem`; we've provided the environment variable for you) and also link in the CUDA driver API (`-lcuda`). We also need to add to the code `#include <nvshmem.h>`[<sup>4</sup>](#footnote4) and `#include <nvshmemx.h>`[<sup>5</sup>](#footnote5). Finally we need to add `-rdc=true` to enable [relocatable device code](https://docs.nvidia.com/cuda/cuda-compiler-driver-nvcc/index.html#using-separate-compilation-in-cuda), a requirement of NVSHMEM.

## Exercise: Use NVSHMEM with MC π Code

For a first exercise with NVSHMEM, address the FIXMEs in [exercises/pi_step1.cpp](exercises/nvshmem_pi_step1.cpp) to have each GPU perform identical work.

Check out the [solution](solutions/nvshmem_pi_step1.cpp) if you get stuck.

In [ ]:
!nvcc -x cu -arch=sm_70 -rdc=true -I $NVSHMEM_HOME/include -L $NVSHMEM_HOME/lib -lnvshmem -lcuda -o nvshmem_pi_step1 exercises/nvshmem_pi_step1.cpp
!nvshmrun -np $NUM_DEVICES ./nvshmem_pi_step1

## Next

You ran your first NVSHMEM program but unfortunately what we did was not very interesting -- every PE did the same work. (Check that this is true by comparing the output from all PEs.) Ideally we would like to distribute the work across PEs and GPUs. In the next notebook we will take our first step in that direction.

Please open the next notebook: [_Monte Carlo Approximation of $\pi$ - NVSHMEM with Distributed Work_](08_MCπ-NVSHMEM-Dist.ipynb).

## Footnotes

<span id="footnote1">1</span>: `nvshmrun` is essentially a symbolic link to the [Hydra](https://wiki.mpich.org/mpich/index.php/Using_the_Hydra_Process_Manager) process manager. However, while in this course we demonstrate the use of NVSHMEM in a standalone style, NVSHMEM is compatible with MPI job launching environments and can also be used for example with [Slurm](https://slurm.schedmd.com/overview.html).

If launching the job with MPI or OpenSHMEM, then a relevant code modification would look like the following. For MPI, we initialize MPI first and then bootstrap NVSHMEM initialization on top of MPI. When shutting down, we finalize NVSHMEM and then finalize MPI.

```cpp
int main() {
    MPI_Init(&argc, &argv);

    nvshmemx_init_attr_t attr;
    MPI_Comm comm = MPI_COMM_WORLD;
    attr.mpi_comm = &comm;

    nvshmemx_init_attr(NVSHMEMX_INIT_WITH_MPI_COMM, &attr);

    // ...

    nvshmem_finalize();

    MPI_Finalize();

    return 0;
}
```

In an OpenSHMEM job, we would instead do

```cpp
int main() {
    shmem_init();

    nvshmemx_init_attr_t attr;
    nvshmemx_init_attr(NVSHMEMX_INIT_WITH_SHMEM, &attr);

    // ...

    nvshmem_finalize();

    shmem_finalize();

    return 0;
}
```

<span id="footnote2">2</span>: NVSHMEM, being an implementation of [OpenSHMEM](http://openshmem.org/site/), shares much of OpenSHMEM's terminology (e.g. PEs) and has very similar APIs. Readers familiar with MPI will recognize that PEs are analogous to MPI ranks.

<span id="footnote3">3</span>: The API `nvshmem_team_my_pe()` is new in NVSHMEM 2.0. See [this blog post](https://developer.nvidia.com/blog/accelerating-nvshmem-2-0-team-based-collectives-using-nccl/) for more.

<span id="footnote4">4</span>: `nvshmem.h` provides OpenSHMEM standard-conforming APIs, which look like `nvshmem_*`.

<span id="footnote5">5</span>: `nvshmemx.h` provides NVIDIA-specific extensions, which look like `nvshmemx_*`.